In [1]:
import pennylane as qml
import numpy as np
from scipy.linalg import expm,sinm,cosm
from numpy.linalg import solve

# Condition Number

In [2]:
def condition_number(matrix):
    eigv,eigc=np.linalg.eig(matrix)
    eigv
    abs_eigv=[]
    for i in eigv:
        if abs(i)!=0:
            abs_eigv.append(abs(i))
        else:
            continue
    return (max(abs_eigv)/min(abs_eigv),min(abs_eigv))

# Function $log_2(\bullet)$ 

In [3]:
def lg(x,y):
    return np.log(y)/np.log(x)

### Lemma 11. Let the function $h(x)$ be defined as

$$
h(x):=\frac{i}{\sqrt{2\pi}}\sum_{j=0}^{J-1}\Delta_{y}\sum_{k=-K}^{K}\Delta_{z}\,z_{k}e^{-z_{k}^{2}/2}e^{-i x y_{j}z_{k}}, 
$$

where $y_{j}:=j\Delta y$  $z_{k}:=k\Delta z.$ for some fixed $J~=~\Theta(\textstyle{\frac{\kappa}{\epsilon}}\log(\kappa/\epsilon))$ , $K\;=\;{\Theta}(\kappa\log(\kappa/\epsilon))$  ,$\Delta_{\mathrm{y}}=$  $\Theta(\epsilon/\sqrt{\log\left(\kappa/\epsilon\right)})$ , and $\Delta_{z}=\Theta((\kappa\sqrt{\log(\kappa/\epsilon)})^{-1})$. Then $h(x)$ is $\epsilon$ -close to $1/x$ on the domain $D_{k}$.

For a given matrix $U$, we could derive the parameters: $\Delta y$,$\Delta z$,$J$ and $K$.

In [4]:
%run ISING_INSPIRED_H.ipynb
def parameter(matrix,epsilon):
    Condition_number=np.linalg.cond(matrix)
    x=Condition_number/epsilon
    Delta_y=epsilon/np.sqrt(lg(2,x))
    Delta_z=1/(Condition_number*np.sqrt(lg(2,x)))
    J_=int((x*lg(2,x))//1+1)
    K_=int((Condition_number*lg(2,x))//1+1)
    return (Delta_y,Delta_z,J_,K_)

# The inverse of matrix

In [5]:
def matrix_inverse(U,delta_y,delta_z,J,K):
    inverse=0
    for j in range(J):
        yj=j*delta_y
        for k in range (K+1):
            zk=k*delta_z
            if k ==0:
                inverse=inverse+delta_y*delta_z*zk*np.exp(-(zk**2)/2)*expm(-1j*U*yj*zk)
            else:
                inverse=inverse-1*delta_y*delta_z*zk*np.exp(-zk**2/2)*expm(1j*U*yj*zk)
                inverse=inverse+delta_y*delta_z*zk*np.exp(-zk**2/2)*expm(-1j*U*yj*zk)
    return (inverse*(1j/np.sqrt(2*np.pi)))

# The error calculate

In [6]:
def error(U,delta_y,delta_z,J,K,b):
    m=matrix_inverse(U,delta_y,delta_z,J,K)
    x=np.dot(m,b)
    b_=np.dot(U,x)
    b1=b_/(np.linalg.norm(b_))
    return(abs(1-np.inner(b1,b)))

# An Example in the Paper

Given a 10-qubit Hermitian matrix $\mathcal{H}=\frac{1}{\zeta}\left( \sum_{j=1}^{10} X_{j}+J \sum_{j=1}^{9} Z_{j} Z_{j+1}+\eta\mathcal{I}\right)$, where $J=0.1$, $\eta=1.37$, and $\zeta=110.6982637623554$. The purpose of selecting these parameters is to ensure that the condition number of the matrix satisfies the given value of 176.6.

The state vector is given by $\ket{b}=H^{\otimes 10}\ket{0}$.

The error tolerance is set to $\epsilon=0.001$.

While it is theoretically possible to solve this strictly through the Fourier Approach, it is evident that substituting the given parameters into the calculations would require an impractically large number of time-evolution unitary operators, making it computationally infeasible with our current resources. As an alternative, we have chosen to simulate this scenario by setting different parameters:

- **Case 1:** $J=2K$ and $J=10K$, where $K=1, 2, 3, 4, 5, 6, 7, 8$. The results are: [0.0009433834750935954, 0.000943383474822701, 0.0009433834736484181, 0.0009433834705396826, 0.0009433834640969474, 0.0009433834525892637, 0.0009433834339735991]

- **Case 2:** $J=2K$ and $J=10K$, where $K=1, 2, 3, 4, 5, 6, 7, 8$. The results are: [0.0009433834744784209, 0.0009433834661043417, 0.0009433834328116397, 0.0009433833478552645, 0.0009433831753996591, 0.000943382871206655, 0.0009433823835013344]

In [ ]:
n=10
H=Ising_inspired_Hamiltonian(n,   0.1,1.37,10*(11.06982637623554)
)
U=qml.matrix(H)
b=np.ones((2**10,2**10))[0]/(np.sqrt(2**10))
epsilons=0.001-

In [8]:
e=[]
for i in range (1,8):
    e.append(error(U,0.001,0.1,2*i,i,b))

In [9]:
e

[0.0009433834750935954,
 0.000943383474822701,
 0.0009433834736484181,
 0.0009433834705396826,
 0.0009433834640969474,
 0.0009433834525892637,
 0.0009433834339735991]

In [10]:
e=[]
for i in range (1,8):
    e.append(error(U,0.001,0.1,10*i,i,b))

In [11]:
e

[0.0009433834744784209,
 0.0009433834661043417,
 0.0009433834328116397,
 0.0009433833478552645,
 0.0009433831753996591,
 0.000943382871206655,
 0.0009433823835013344]